In [1]:
import numpy as np
import pandas as pd

In [5]:
# Read from all kinds of dayaset
trip15q2 = pd.read_csv('./indegotrip/indego-trips-2015-q2.csv')
trip15q3 = pd.read_csv('./indegotrip/indego-trips-2015-q3.csv')
trip15q4 = pd.read_csv('./indegotrip/indego-trips-2015-q4.csv')
trip16q1 = pd.read_csv('./indegotrip/indego-trips-2016-q1.csv')
trip16q2 = pd.read_csv('./indegotrip/indego-trips-2016-q2.csv')
trip16q3 = pd.read_csv('./indegotrip/indego-trips-2016-q3.csv')
trip16q4 = pd.read_csv('./indegotrip/indego-trips-2016-q4.csv')
trip17q1 = pd.read_csv('./indegotrip/indego-trips-2017-q1.csv')
trip17q2 = pd.read_csv('./indegotrip/indego-trips-2017-q2.csv')
trip17q3 = pd.read_csv('./indegotrip/indego-trips-2017-q3.csv')
trip17q4 = pd.read_csv('./indegotrip/indego-trips-2017-q4.csv')
trip18q1 = pd.read_csv('./indegotrip/indego-trips-2018-q1.csv')
trip18q2 = pd.read_csv('./indegotrip/indego-trips-2018-q2.csv')
trip18q3 = pd.read_csv('./indegotrip/indego-trips-2018-q3.csv')
trip18q4 = pd.read_csv('./indegotrip/indego-trips-2018-q4.csv')
trip19q1 = pd.read_csv('./indegotrip/indego-trips-2019-q1.csv')

C:\Users\Kevin Long\Anaconda3\envs\musa\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
# change all separate dataset into similar form and get a combined dataset
dataset1 = [trip15q2, trip15q3, trip15q4, trip16q1, trip16q2, trip16q3, trip16q4, trip17q1]
dataset1 = pd.concat(dataset1)
df1 = dataset1.copy()
df1['duration']  = df1['duration'] / 60

dataset2 = [trip17q2,trip17q3,trip17q4,trip18q1,trip18q2]
dataset2 = pd.concat(dataset2)
df2 = dataset2.copy()

dataset3 = [trip18q3,trip18q4,trip19q1]
dataset3 = pd.concat(dataset3)
df3 = dataset3.copy()
df3 = df3[df3.columns[:-1]]

df1.columns, df2.columns = df3.columns, df3.columns
df = pd.concat([df1,df2,df3])
df.head()

# The combined dataset is around 2.4 million trips, which satisfy the requirement of final project 

In [71]:
# Getting data popup 
from datetime import datetime, timedelta
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

In [75]:
df.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type
0,4001912,1.0,2015-04-23 07:44:00,2015-04-23 07:45:00,3046.0,39.9501,-75.1447,3046,39.9501,-75.1447,3350,30,Round Trip,Indego30
1,4001916,1.0,2015-04-23 07:45:00,2015-04-23 07:46:00,3046.0,39.9501,-75.1447,3046,39.9501,-75.1447,3350,30,Round Trip,Indego30
2,4001918,2.0,2015-04-23 07:46:00,2015-04-23 07:48:00,3057.0,39.9644,-75.18,3057,39.9644,-75.18,3282,0,Round Trip,Walk-up
3,4001938,1.0,2015-04-23 10:13:00,2015-04-23 10:14:00,3004.0,39.9538,-75.1637,3004,39.9538,-75.1637,3350,30,Round Trip,Indego30
4,4001955,3.0,2015-04-23 10:14:00,2015-04-23 10:17:00,3004.0,39.9538,-75.1637,3004,39.9538,-75.1637,2716,30,Round Trip,Indego30


In [81]:
df_lost = df[pd.isnull(df['end_lat'])]

In [165]:
def datelist(beginDate, endDate):
    date_list = [datetime.strftime(x, '%Y-%m-%d %H') for x in list(pd.date_range(start= beginDate, end = endDate, freq = '1H'))]
    return date_list

In [205]:
def get_lost(df, datelist):
    combine = {}
    lost_trip = {}
    lost_trip_location = {}
    lost_time = {}
    
    lost_bike = []
    lost_bike_combine = {}
    
    
    missing_bike = {}
    missing_bike_combine = {}
    
    missing_combine = {}
    for stamp in datelist:
        missing_trip = {}
        
        # update lost hourse for everybikes in missing list
        if len(lost_time) != 0:
            lost_time = {key_: val_+ 1 for key_, val_ in lost_time.items()}
        
        # Iterate only trips within this time span
        df_trip = df[(df['start_time'] >= datetime.strptime(stamp,'%Y-%m-%d %H')) & (df['start_time'] < (datetime.strptime(stamp,'%Y-%m-%d %H') + timedelta(hours=1)))]
        
        
        
        for index, single_trip in df_trip.iterrows():
            if pd.isnull(single_trip['end_lat']):
                lost_time[single_trip['bike_id']] = lost_time.get(single_trip['bike_id'], 0) + 1
                missing_trip.update({single_trip['bike_id']:{'lng':single_trip['start_lon'], 'lat':single_trip['start_lat']}})
                lost_trip.update({single_trip['bike_id']: single_trip['trip_id']})
                lost_trip_location.update({single_trip['bike_id']: {'lng':single_trip['start_lon'], 'lat':single_trip['start_lat']}})
                
            # If any missing bikes re-appears in another trip, and this new trip does have ended somewhere, 
            if (lost_time.get(single_trip['bike_id']) != None) & (single_trip['trip_id'] != lost_trip.get(single_trip['bike_id'])) & pd.notnull(single_trip['end_lat']):
                lost_time.pop(single_trip['bike_id'])
                lost_trip.pop(single_trip['bike_id'])
                lost_trip_location.pop(single_trip['bike_id'])
                
        # Getting all lost bike (last trip lacks end coordinate and haven't been re-appear in over 7 days)
        if len(lost_time) != 0:
            for key, val in lost_time.items():
                if val >= 168:
                    lost_bike.append(key)
                    missing_bike.update({key:lost_trip_location.get(key)})
                                       
        # Generate data for plotting based on timestamp
        lost_bike_combine.update({stamp:list(set(lost_bike))})  # Lost bike and their lost hour
        missing_bike_combine.update({stamp: missing_bike.copy()}) # 
        missing_combine.update({stamp:missing_trip.copy()})
        combine.update({stamp:lost_time.copy()})
        
    return combine, missing_combine,lost_bike_combine, missing_bike_combine          

In [212]:
# Running the function from 2016 to 2018 (which should be 365*3*24 rows of data used for visualization)
a,b,c,d = get_lost(df, datelist('2016-01-01', '2019-01-01'))  

In [ ]:
# Generating four dataset, all of which controlled by timestamp
time_tracking = pd.DataFrame(a)
time_tracking.to_csv('vis2_lost_time.csv')
missing_trips = pd.DataFrame(b)
missing_trips.to_csv('vis1_missing_popup.csv')

In [216]:
lost_locations = pd.DataFrame(d)
lost_locations.to_csv('vis4_lost_location.csv')

In [251]:
lost_bike_num = {}
lost_bike = {}
for key,val in c.items():
    lost_bike_num.update({key:[len(val)]})
    lost = {0:1}
    for i in val:
        lost.update({i:1})
    lost_bike.update({key:lost})

In [243]:
lost_bike_num = pd.DataFrame(lost_bike_num)
lost_bike_num.to_csv('vis3_stats.csv')

In [255]:
import json
with open('vis3_lost_bike.json', 'w') as fp:
    json.dump(lost_bike, fp)